In [1]:
import uproot
import numpy as np
import numba
import awkward1 as ak
import os
import optparse

from data import getData
from plot_utils import plotHist, plotCollection, plotEfficiency, combinePDFs
from truth_utils import truth_link, dr_match

In [3]:
cms_events = getData()

In [4]:
truth_builder = truth_link(cms_events['genParticles'], ak.ArrayBuilder())
truth_extension = truth_builder.snapshot()
ele_mask = np.abs(cms_events['genParticles']['pdgId']) == 11
first_mask = truth_extension['isFirst']
last_mask = np.abs(cms_events['genParticles']['status']) == 1 # equivalent to last                                                                                                            
z_mask = truth_extension['motherPdgId'] == 23
n2_mask = truth_extension['motherPdgId'] == 1000023
gen_ele_mask = ele_mask & (z_mask | n2_mask) & last_mask
truth_electrons = cms_events['genParticles'][gen_ele_mask]

In [13]:
#reco_electrons = cms_events['softElectrons']
reco_electrons = cms_events['electrons']
reco_cuts = {
    "all": reco_electrons.pt > -1,
#    "looseMVA": reco_electrons.mvaId > 0,
#    "tightMVA": reco_electrons.mvaId > 5,
    }


In [14]:
cut_name="all"
reco_mask = reco_cuts[cut_name]
signal_electrons = reco_electrons[reco_mask]

match_builder = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=False)
match_extension = match_builder.snapshot()
matching_truth_mask = match_extension.truth_to_reco_index >= 0
# matching_reco_mask = match_extension.truth_to_reco_index[matching_truth_mask]                                                                                                      

match_builderR = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=True)
match_extensionR = match_builderR.snapshot()
matching_reco_mask = match_extensionR.reco_to_truth_index >= 0
nonmatching_reco_mask = match_extensionR.reco_to_truth_index < 0

matched_truth = truth_electrons[matching_truth_mask]
unmatched_truth = truth_electrons[~matching_truth_mask]
matched_reco = signal_electrons[matching_reco_mask]
unmatched_reco = signal_electrons[~matching_reco_mask]
# unmatched_reco = signal_electrons[nonmatching_reco_mask]      

In [29]:
nprint=10
print(ak.num(truth_electrons)[:nprint])
print(ak.num(reco_electrons)[:nprint])
print(ak.to_list(truth_electrons.eta)[:nprint])
print(ak.to_list(reco_electrons.eta)[:nprint])
print(ak.to_list(truth_electrons.phi)[:nprint])
print(ak.to_list(reco_electrons.phi)[:nprint])

[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]
[0, 0, 1, 0, 1, 1, 0, 1, 1, 1]
[[0.4287109375, -3.265625], [], [-1.88671875, -1.625], [], [], [0.353515625, -0.7578125], [], [-1.8203125, -1.70703125], [], [-0.21044921875, -1.359375]]
[[], [], [-1.887939453125], [], [-1.7236328125], [0.35333251953125], [], [-1.708984375], [-0.9852294921875], [-0.21038818359375]]
[[-2.90625, -1.3671875], [], [-2.140625, 1.83203125], [], [], [-0.650390625, 0.654296875], [], [1.8515625, -0.3037109375], [], [-0.560546875, -0.11376953125]]
[[], [], [-2.1416015625], [], [0.35821533203125], [-0.65087890625], [], [-0.30596923828125], [-2.6064453125], [-0.5615234375]]


In [30]:
print(ak.num(matched_truth.pt)[:nprint])
print(ak.to_list(matched_truth.pt)[:nprint])

[0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[[], [], [4.96875], [], [], [10.09375], [], [6.984375], [], [9.46875]]


In [31]:
print(ak.num(unmatched_truth.pt)[:nprint])
print(ak.to_list(unmatched_truth.pt)[:nprint])

[2, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[[2.8671875, 0.74609375], [], [2.265625], [], [], [1.453125], [], [3.4140625], [], [2.96875]]


In [32]:
print(ak.num(matched_reco.pt)[:nprint])
print(ak.to_list(matched_reco.pt)[:nprint])

[0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
[[], [], [4.989854335784912], [], [], [10.219705581665039], [], [8.438362121582031], [], [9.532796859741211]]


In [33]:
print(ak.num(unmatched_reco.pt)[:nprint])
print(ak.to_list(unmatched_reco.pt)[:nprint])

[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[[], [], [], [], [9.597420692443848], [], [], [], [14.239029884338379], []]


In [34]:
print((ak.num(matched_truth.pt)+ak.num(unmatched_truth.pt))[:nprint])
print(ak.num(truth_electrons)[:nprint])

[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]
[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]


In [35]:
print((ak.num(matched_reco.pt)+ak.num(unmatched_reco.pt))[:nprint])
print(ak.num(reco_electrons)[:nprint])

[0, 0, 1, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 0, 1, 1, 1]


Now check that the contents are OK

In [40]:
print((matched_truth.pt)[:nprint])
print((unmatched_truth.pt)[:nprint])
print((truth_electrons.pt)[:nprint])

[[], [], [4.97], [], [], [10.1], [], [6.98], [], [9.47]]
[[2.87, 0.746], [], [2.27], [], [], [1.45], [], [3.41], [], [2.97]]
[[2.87, 0.746], [], [4.97, 2.27], [], [], ... [], [3.41, 6.98], [], [9.47, 2.97]]
